#### Mapper Configuration Options with Declarative

With _all_ mapping forms, the `mapping of the class` is __configured through parameters__ that become part of the `Mapper` object. The _function_ which __ultimately receives these arguments__ is the `mapper()` function, and are _delivered_ to it from one of the _front-facing mapping functions_ defined on the `registry` object.

For the _declarative form_ of mapping, mapper arguments are specified using the `__mapper_args__` declarative class variable, which is a __dictionary that is passed as keyword arguments__ to the `mapper()` function.

In [1]:
from sqlalchemy import Column, String, Integer, DateTime, select
from sqlalchemy.orm import declared_attr, declarative_base
from datetime import datetime

In [2]:
Base = declarative_base()

##### Map Specific Primary Key Columns

The example below illustrates _Declarative-level settings_ for the `mapper.primary_key` parameter, which __establishes__ particular columns as part of what the `ORM` _should consider_ to be a __primary key for the class__, _independently_ of schema-level `primary key constraints`.

In [3]:
class GroupUsers(Base):
    __tablename__ = "group_users"
    
    user_id = Column(String(40))
    group_id = Column(String(40))
    
    __mapper_args__ = {"primary_key": [user_id, group_id]}

##### Version ID Column

The example below illustrates _Declarative-level settings_ for the `mapper.version_id_col` and `mapper.version_id_generator` parameters, which __configure an ORM-maintained version counter__ that is _updated and checked_ within the unit of work `flush process`.

In [4]:
class Widget(Base):
    __tablename__ = "widgets"
    
    id = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False)
    
    __mapper_args__ = {
        "version_id_col": timestamp,
        "version_id_generator": lambda v: datetime.now(),
    }

##### Single Table Inheritance

The example below illustrates _Declarative-level settings_ for the `mapper.polymorphic_on` and `mapper.polymorphic_identity` parameters, which are used when __configuring a single-table inheritance mapping__.

In [5]:
class Person(Base):
    __tablename__ = "person"
    
    person_id = Column(Integer, primary_key=True)
    type = Column(String, nullable=False)
    
    __mapper_args__ = {
        "polymorphic_on": type,
        "polymorphic_identity": "person",
    }

In [6]:
class Employee(Person):
    __mapper_args__ = {
        "polymorphic_identity": "employee",
    }

##### Constructing mapper arguments dynamically

The `__mapper_args__` dictionary _may be generated_ from a __class-bound descriptor method__ rather than from a _fixed dictionary_ by making use of the `declared_attr()` construct. This is __useful to create arguments for mappers that are programmatically derived__ from the _table configuration or other aspects_ of the `mapped class`. A __dynamic `__mapper_args__` attribute__ will typically be useful when using a `Declarative Mixin` or `abstract base class`.

For example, to __omit from the mapping__ _any columns that have a special_ `Column.info` value, a _mixin_ can use a `__mapper_args__` method that __scans for these columns__ from the `cls.__table__` attribute and passes them to the `mapper.exclude_properties` collection.

In [7]:
class ExcludeColsWFlag:
    @declared_attr
    def __mapper_args__(cls):
        return {
            "exclude_properties": [
                column.key for column in cls.__table__.c
                if column.info.get("exclude", False)
            ]
        }

In [8]:
class Data(ExcludeColsWFlag, Base):
    __tablename__ = "data"
    
    id = Column(Integer, primary_key=True)
    data = Column(String)
    
    not_needed = Column(String, info={"exclude": True})

Above, the `ExcludeColsWFlag` _mixin_ provides a __per-class `__mapper_args__` hook__ that will _scan_ for `Column` objects that __include the `key/value` `"exclude": True`__ passed to the `Column.info` parameter, and then add their string `"key"` name to the `mapper.exclude_properties` collection which will __prevent the resulting `Mapper` from considering these columns__ for any SQL operations.